<a href="https://colab.research.google.com/github/valerio-unifei/UNIFEI-IA-Aulas/blob/main/UNIFEI_IA_Decomposi%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Decomposição da Matriz

## Dataset

In [ ]:
import pandas as pd
from sklearn import datasets

breast = datasets.load_breast_cancer()
pd.DataFrame(breast.data,columns=breast.feature_names)

## Dimensões e Qualidade da Generalização

In [ ]:
import time
from sklearn import decomposition, neural_network, model_selection

resultados = []

y = breast.target

for componentes in range(5, breast.data.shape[1],5):

  decomposicoes = (
      (decomposition.PCA(n_components=componentes, random_state=42),'PCA'),
      (decomposition.TruncatedSVD(n_components=componentes,random_state=42),'LSA'),
      (decomposition.FactorAnalysis(n_components=componentes,random_state=42),'FA'),
      (decomposition.FastICA(n_components=componentes,random_state=42),'ICA'),
      (decomposition.NMF(n_components=componentes,random_state=42,init='random'),'NMF'),
      (decomposition.LatentDirichletAllocation(n_components=componentes,random_state=42),'LDA'),
    )

  for dcp, nome in decomposicoes:
    t0 = time.time()
    X = dcp.fit_transform(breast.data,y)
    tempo = time.time() - t0

    clf = neural_network.MLPClassifier(hidden_layer_sizes=(100,50,),random_state=42)
    res = model_selection.cross_validate(clf, X, y, cv=5,n_jobs=-1,scoring=['accuracy','f1_macro','average_precision'])
    clf.fit(X,y)

    linha = []
    linha.append(componentes)
    linha.append(nome)
    linha.append(tempo)
    linha.append(sum(res['fit_time'])/5)
    linha.append(sum(res['test_accuracy'])/5)
    linha.append(sum(res['test_f1_macro'])/5)
    linha.append(sum(res['test_average_precision'])/5)
    resultados.append(linha)

pd.DataFrame(resultados, columns=['Atributos','Técnica','Tempo Redução','Tempo Treinamento','Acurácia','F1','Precisão Média'])